In [1]:
import pandas as pd 
from data_clean import convert_to_int
import requests
from keys import KEY_ONE
import numpy as np

# import theme df, join merged_df_three with theme to get star wars theme. 
#add code to not include first columns
inv_df = pd.read_csv('.\CSVs\inventories.csv')
inv_parts_df = pd.read_csv('.\CSVs\inventory_parts.csv')
set_df = pd.read_csv('.\CSVs\sets.csv')
parts_df = pd.read_csv('.\CSVs\parts.csv')

#group by part set ID, count pieces to find max
# for set ID group by part #, pieces, find max


In [2]:
merged_df_one = inv_df.merge(inv_parts_df, how='left', left_on='id', right_on='inventory_id')
merged_df_one.drop(columns='Unnamed: 0_y', inplace=True)
merged_df_two = merged_df_one.merge(set_df, how='left', left_on='set_num', right_on='set_num')
merged_df_two.drop(columns='Unnamed: 0', inplace=True)
merged_df_three = merged_df_two.merge(parts_df, how='left', left_on='part_num', right_on='part_num')
merged_df_three.drop(columns='Unnamed: 0', inplace=True)


In [3]:
# Convert theme to int
print(merged_df_three.shape)

merged_df_three['quantity'] = merged_df_three['quantity'].astype(pd.Int64Dtype())
merged_df_three['color_id'] = merged_df_three['color_id'].astype(pd.Int64Dtype())
merged_df_three['theme_id'] = merged_df_three['theme_id'].astype(pd.Int64Dtype())


# convert_to_int(merged_df_three['quantity'])
# convert_to_int(merged_df_three['color_id'])
# convert_to_int(merged_df_three['theme_id'])

sw_theme_ids = [18, 158, 171, 209, 261]
merged_df_three = merged_df_three[merged_df_three['theme_id'].isin(sw_theme_ids)]

print(merged_df_three.shape)



(1045404, 16)
(86621, 16)


In [4]:

piece_count = merged_df_three.groupby(['set_num'])['quantity'].sum()
convert_to_int(piece_count)

max_count = piece_count.idxmax()
print(max_count)


75192-1


In [5]:
mask = merged_df_three[merged_df_three['set_num'] != max_count].index
merged_df_three.drop(mask, inplace=True)


In [6]:
top_ten_parts = merged_df_three['quantity'].nlargest(n=10, keep='first')
mask_two = merged_df_three[merged_df_three['quantity'].isin(top_ten_parts)]
mask_two.reset_index(drop=True, inplace=True)


In [7]:
part_num_list =[]
part_color_list = []

for item in mask_two['part_num']:
    part_num_list.append(item)
for item in mask_two['color_id']:
    part_color_list.append(str(item))
num_color_zip = zip(part_num_list,part_color_list)

url_list = []
for num, color in num_color_zip:
    response = requests.get(f'https://rebrickable.com/api/v3/lego/parts/{num}/colors/{color}?key={KEY_ONE}')
    data = response.json()
    url_list.append(str(data['part_img_url']))

url_df = pd.DataFrame({'part_num' : part_num_list,
                       'URL' : url_list})

url_df['part_num'] = url_df['part_num'].astype(str)
url_df['URL'] = url_df['URL'].astype(str)

mask_two = mask_two.merge(url_df, how='left', left_on='part_num', right_on='part_num')
mask_two
    

,Unnamed: 0_x,id,version,set_num,inventory_id,part_num,color_id,quantity,is_spare,name_x,year,theme_id,num_parts,name_y,part_cat_id,part_material,URL
0,11009,19670,1,75192-1,19670.0,15573,72,75,f,Millennium Falcon,2017.0,171,7541.0,Plate Special 1 x 2 with 1 Stud with Groove an...,9.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
1,11009,19670,1,75192-1,19670.0,15712,72,139,f,Millennium Falcon,2017.0,171,7541.0,Tile Special 1 x 1 with Clip with Rounded Edges,15.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
2,11009,19670,1,75192-1,19670.0,2412b,0,102,f,Millennium Falcon,2017.0,171,7541.0,Tile Special 1 x 2 Grille with Bottom Groove,15.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
3,11009,19670,1,75192-1,19670.0,2780,0,269,f,Millennium Falcon,2017.0,171,7541.0,Technic Pin with Friction Ridges Lengthwise an...,53.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
4,11009,19670,1,75192-1,19670.0,3021,71,105,f,Millennium Falcon,2017.0,171,7541.0,Plate 2 x 3,14.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
5,11009,19670,1,75192-1,19670.0,3023,71,73,f,Millennium Falcon,2017.0,171,7541.0,Plate 1 x 2,14.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
6,11009,19670,1,75192-1,19670.0,3023,71,73,f,Millennium Falcon,2017.0,171,7541.0,Plate 1 x 2,14.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
7,11009,19670,1,75192-1,19670.0,3023,28,243,f,Millennium Falcon,2017.0,171,7541.0,Plate 1 x 2,14.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
8,11009,19670,1,75192-1,19670.0,3023,28,243,f,Millennium Falcon,2017.0,171,7541.0,Plate 1 x 2,14.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...
9,11009,19670,1,75192-1,19670.0,32054,71,75,f,Millennium Falcon,2017.0,171,7541.0,Technic Pin Long with Friction Ridges Lengthwi...,53.0,Plastic,https://cdn.rebrickable.com/media/parts/elemen...


In [8]:
for item in url_list:
    img_data = requests.get(item).content
    head, sep, tail = item.partition('elements/')
    with open(f'./JPGs/{tail}', 'wb') as handler:
        handler.write(img_data)